In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools

### Parameters

In [2]:
Random.seed!(123)
# 设置参数
N = 3   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2

# 预设数据（根据你的实际问题替换这些值）
u = rand(N_u)        # u 向量 ∈ ℝ^N
A = rand(N,N_u)      # a_n ∈ ℝ^N
# B = rand(N,N)        # b_n ∈ ℝ^N
p_dag = rand(N, K)   # p_{nk}^† 数据矩阵

3×10 Matrix{Float64}:
 0.0802658  0.654101   0.271636  0.88463   …  0.862761  0.387374  0.875555
 0.0490284  0.256433   0.688575  0.810734     0.158312  0.771916  0.264568
 0.915866   0.0862789  0.789013  0.257922     0.5403    0.377383  0.901877

In [3]:
function generate_strictly_row_diagonally_dominant(n::Int, max_offdiag)
    Mat = zeros(n, n)
    for i in 1:n
        # 在非对角线上生成随机数 [-max_offdiag, max_offdiag]
        off_diag = rand(n) .* (2max_offdiag) .- max_offdiag
        off_diag[i] = 0  # 避免给自己赋值两次

        # 计算非对角元素绝对值之和
        sum_offdiag = sum(abs, off_diag)

        # 设置对角元，使其严格大于其他元素之和
        diag_value = sum_offdiag + rand() * max_offdiag + 1e-3  # 加小量保证严格性

        Mat[i, :] = abs.(off_diag)
        Mat[i, i] = -abs.(diag_value)
    end
    return Mat
end

generate_strictly_row_diagonally_dominant (generic function with 1 method)

In [4]:
B = generate_strictly_row_diagonally_dominant(N, 1.0)

3×3 Matrix{Float64}:
 -1.84667    0.464476   0.793606
  0.618866  -1.71233    0.717049
  0.353414   0.841769  -1.89217

In [5]:
function compute_oof(X_given, A, B, u, p_dag)
    price = sum(X_given .* p_dag,dims=2)
    utilities = ones(N+1)
    utilities[1:N] = exp.(vec(A * u + B * price))

    prob = ones(N)
    rev = ones(N)
    for i in 1:N
        prob[i] = utilities[i] / sum(utilities)
        rev[i] = prob[i] * price[i]
    end
    # println("prob = ", prob)
    # println("rev = ", rev)
    # println("total rev = ", sum(rev))
    return sum(rev)
end

compute_oof (generic function with 1 method)

In [6]:
rev_opt = 0
price_opt = zeros(N)

rev_min = 1000
price_min = zeros(N)
for i in 1:K
    p1 = p_dag[1,i]
    for j in 1:K
        p2 = p_dag[2,j]
        for l in 1:K
            p3 = p_dag[3,l]
            price = [p1,p2,p3]
            utilities = ones(N+1)
            utilities[1:N] = exp.(vec(A * u + B * price))
        
            prob = zeros(N)
            rev = zeros(N)
            for i in 1:N
                prob[i] = utilities[i] / sum(utilities)
                rev[i] = prob[i] * price[i]
            end
            if sum(rev) > rev_opt
                rev_opt = sum(rev)
                price_opt = price
            end

            if sum(rev) < rev_min
                rev_min = sum(rev)
                price_min = price
            end
        end
    end
end
println("Optimal revenue = ", rev_opt)
println("Optimal price = ", price_opt)

println("Minimum revenue = ", rev_min) 
println("Minimum price = ", price_min)

Optimal revenue = 0.6883497210244508
Optimal price = [0.884629777785838, 0.9111498653643074, 0.9158663552785268]
Minimum revenue = 0.06325396744728443
Minimum price = [0.08026576094597515, 0.04902841674350844, 0.08627888341903334]


### Estimate-then-price

In [28]:
tau = 0

0

In [29]:
model = Model(Mosek.Optimizer)
# set_attribute(model, "QUIET", true)
# model = Model(COPT.ConeOptimizer)
# 定义变量
@variable(model, rho_0 >= 0)                       # ρ₀ ≥ 0
@variable(model, rho[1:N] >= 0)                    # ρ_n ≥ 0
@variable(model, v_sigma[1:N_u])                   # ς = ρ₀ * u
@variable(model, v_phi[1:N])                       # φ_n

@variable(model, v_theta[1:N,1:N_u])                   # ς = ρ₀ * u
@variable(model, v_pi[1:N,1:N])  

@variable(model, Y[1:N, 1:K])                      # y_{nk}
@variable(model, Z[1:N, 1:K])                      # z_{nk}
@variable(model, X[1:N, 1:K], Bin)                 # x_{nk} ∈ {0,1}

@variable(model, Z2[1:N,1:N,1:K])                      # z_{nk}


3×3×10 Array{VariableRef, 3}:
[:, :, 1] =
 Z2[1,1,1]  Z2[1,2,1]  Z2[1,3,1]
 Z2[2,1,1]  Z2[2,2,1]  Z2[2,3,1]
 Z2[3,1,1]  Z2[3,2,1]  Z2[3,3,1]

[:, :, 2] =
 Z2[1,1,2]  Z2[1,2,2]  Z2[1,3,2]
 Z2[2,1,2]  Z2[2,2,2]  Z2[2,3,2]
 Z2[3,1,2]  Z2[3,2,2]  Z2[3,3,2]

[:, :, 3] =
 Z2[1,1,3]  Z2[1,2,3]  Z2[1,3,3]
 Z2[2,1,3]  Z2[2,2,3]  Z2[2,3,3]
 Z2[3,1,3]  Z2[3,2,3]  Z2[3,3,3]

[:, :, 4] =
 Z2[1,1,4]  Z2[1,2,4]  Z2[1,3,4]
 Z2[2,1,4]  Z2[2,2,4]  Z2[2,3,4]
 Z2[3,1,4]  Z2[3,2,4]  Z2[3,3,4]

[:, :, 5] =
 Z2[1,1,5]  Z2[1,2,5]  Z2[1,3,5]
 Z2[2,1,5]  Z2[2,2,5]  Z2[2,3,5]
 Z2[3,1,5]  Z2[3,2,5]  Z2[3,3,5]

[:, :, 6] =
 Z2[1,1,6]  Z2[1,2,6]  Z2[1,3,6]
 Z2[2,1,6]  Z2[2,2,6]  Z2[2,3,6]
 Z2[3,1,6]  Z2[3,2,6]  Z2[3,3,6]

[:, :, 7] =
 Z2[1,1,7]  Z2[1,2,7]  Z2[1,3,7]
 Z2[2,1,7]  Z2[2,2,7]  Z2[2,3,7]
 Z2[3,1,7]  Z2[3,2,7]  Z2[3,3,7]

[:, :, 8] =
 Z2[1,1,8]  Z2[1,2,8]  Z2[1,3,8]
 Z2[2,1,8]  Z2[2,2,8]  Z2[2,3,8]
 Z2[3,1,8]  Z2[3,2,8]  Z2[3,3,8]

[:, :, 9] =
 Z2[1,1,9]  Z2[1,2,9]  Z2[1,3,9]
 Z2[2,1,9]  Z2[2,2,9]  Z2[2,3

In [30]:
# 1. 总和约束
@constraint(model, rho_0 + sum(rho) == 1)

rho_0 + rho[1] + rho[2] + rho[3] = 1

In [31]:
for n in 1:N
    @constraint(model, [A[n,:]' * v_sigma + B[n,:]' * v_phi,rho_0,rho[n]] in MOI.ExponentialCone())
end

In [32]:
# 2. ς = ρ₀ * u
@constraint(model, v_sigma .== rho_0 .* u)

2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -0.521213795535383 rho_0 + v_sigma[1] = 0
 -0.5868067574533484 rho_0 + v_sigma[2] = 0

In [33]:
# 3. φ_n = ∑_k z_{nk} * p_{nk}^†
@constraint(model, v_phi .== sum(Z .* p_dag, dims=2))

3×1 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 v_phi[1] - 0.08026576094597515 Z[1,1] - 0.6541013048231016 Z[1,2] - 0.27163601818691985 Z[1,3] - 0.884629777785838 Z[1,4] - 0.43851795532571036 Z[1,5] - 0.6943166776232444 Z[1,6] - 0.5320515448896448 Z[1,7] - 0.8627612916760001 Z[1,8] - 0.38737406225356574 Z[1,9] - 0.8755549995596186 Z[1,10] = 0
 v_phi[2] - 0.04902841674350844 Z[2,1] - 0.25643321529948804 Z[2,2] - 0.6885748828439957 Z[2,3] - 0.8107339122926914 Z[2,4] - 0.3738130998598468 Z[2,5] - 0.17965931850740957 Z[2,6] - 0.9111498653643074 Z[2,7] - 0.15831162891445483 Z[2,8] - 0.7719163558447772 Z[2,9] - 0.26456814036193066 Z[2,10] = 0
 v_phi[3] - 0.9158663552785268 Z[3,1] - 0.08627888341903334 Z[3,2] - 0.7890131811814325 Z[3,3] - 0.2579219565123101 Z[3,4] - 0.4506313683009061 Z[3,5] - 0.6494206927359497 Z[3,6] - 0.800398894410319 Z[3,7] - 0.5403003932326621 Z[3,8] - 0.

In [34]:
# 4. 辅助变量上下界
for n in 1:N
    for k in 1:K
        # y_{nk} bounds
        @constraint(model, 0 <= Y[n, k])
        @constraint(model, Y[n, k] <= X[n, k])
        @constraint(model, Y[n, k] >= rho[n] - (1 - X[n, k]))
        @constraint(model, Y[n, k] <= rho[n])
    end
end

In [35]:
# 4. 辅助变量上下界
for n in 1:N
    for k in 1:K
        # z_{nk} bounds
        @constraint(model, 0 <= Z[n, k])
        @constraint(model, Z[n, k] <= X[n, k])
        @constraint(model, Z[n, k] >= rho_0 - (1 - X[n, k]))
        @constraint(model, Z[n, k] <= rho_0)
    end
end

In [36]:
@constraint(model, sum(X,dims=2) .== 1)

3×1 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 X[1,1] + X[1,2] + X[1,3] + X[1,4] + X[1,5] + X[1,6] + X[1,7] + X[1,8] + X[1,9] + X[1,10] = 1
 X[2,1] + X[2,2] + X[2,3] + X[2,4] + X[2,5] + X[2,6] + X[2,7] + X[2,8] + X[2,9] + X[2,10] = 1
 X[3,1] + X[3,2] + X[3,3] + X[3,4] + X[3,5] + X[3,6] + X[3,7] + X[3,8] + X[3,9] + X[3,10] = 1

In [37]:
for n in 1:N
    @constraint(model, [-A[n,:]' * v_theta[n,:] - B[n,:]' * v_pi[n,:], rho[n], rho_0] in MOI.ExponentialCone())
end

In [ ]:
# @constraint(model,sum(v_theta, dims=1)[1,:] .+ v_sigma .- u .== 0)

2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 v_sigma[1] + v_theta[1,1] + v_theta[2,1] + v_theta[3,1] = 0.521213795535383
 v_sigma[2] + v_theta[1,2] + v_theta[2,2] + v_theta[3,2] = 0.5868067574533484

In [38]:
# 2. ς = ρ₀ * u
for n in 1:N
    @constraint(model, v_theta[n,:] .== rho[n] .* u)
end

In [39]:
# 3. φ_n = ∑_k z_{nk} * p_{nk}^†
for n in 1:N
    @constraint(model, v_pi[n,:] .== sum(Z2[n,:,:] .* p_dag, dims=2))
end

In [ ]:
# @constraint(model,sum(v_pi, dims=1)[1,:] .+ v_phi .- sum(X .* p_dag, dims=2) .== 0)

3×1 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 v_phi[1] + v_pi[1,1] + v_pi[2,1] + v_pi[3,1] - 0.08026576094597515 X[1,1] - 0.6541013048231016 X[1,2] - 0.27163601818691985 X[1,3] - 0.884629777785838 X[1,4] - 0.43851795532571036 X[1,5] - 0.6943166776232444 X[1,6] - 0.5320515448896448 X[1,7] - 0.8627612916760001 X[1,8] - 0.38737406225356574 X[1,9] - 0.8755549995596186 X[1,10] = 0
 v_phi[2] + v_pi[1,2] + v_pi[2,2] + v_pi[3,2] - 0.04902841674350844 X[2,1] - 0.25643321529948804 X[2,2] - 0.6885748828439957 X[2,3] - 0.8107339122926914 X[2,4] - 0.3738130998598468 X[2,5] - 0.17965931850740957 X[2,6] - 0.9111498653643074 X[2,7] - 0.15831162891445483 X[2,8] - 0.7719163558447772 X[2,9] - 0.26456814036193066 X[2,10] = 0
 v_phi[3] + v_pi[1,3] + v_pi[2,3] + v_pi[3,3] - 0.9158663552785268 X[3,1] - 0.08627888341903334 X[3,2] - 0.7890131811814325 X[3,3] - 0.2579219565123101 X[3,4] - 0.450

In [40]:
# 4. 辅助变量上下界
for n in 1:N
    for j in 1:N
        for k in 1:K
            # z_{nk} bounds
            @constraint(model, 0 <= Z2[n,j, k])
            @constraint(model, Z2[n,j,k] <= X[j, k])
            @constraint(model, Z2[n,j,k] >= rho[n] - (1 - X[j, k]))
            @constraint(model, Z2[n,j, k] <= rho[n])
        end
    end
end

In [41]:
@objective(model, Max,sum(Y .* p_dag) - tau * sum(rho))

0.08026576094597515 Y[1,1] + 0.04902841674350844 Y[2,1] + 0.9158663552785268 Y[3,1] + 0.6541013048231016 Y[1,2] + 0.25643321529948804 Y[2,2] + 0.08627888341903334 Y[3,2] + 0.27163601818691985 Y[1,3] + 0.6885748828439957 Y[2,3] + 0.7890131811814325 Y[3,3] + 0.884629777785838 Y[1,4] + 0.8107339122926914 Y[2,4] + 0.2579219565123101 Y[3,4] + 0.43851795532571036 Y[1,5] + 0.3738130998598468 Y[2,5] + 0.4506313683009061 Y[3,5] + 0.6943166776232444 Y[1,6] + 0.17965931850740957 Y[2,6] + 0.6494206927359497 Y[3,6] + 0.5320515448896448 Y[1,7] + 0.9111498653643074 Y[2,7] + 0.800398894410319 Y[3,7] + 0.8627612916760001 Y[1,8] + 0.15831162891445483 Y[2,8] + 0.5403003932326621 Y[3,8] + 0.38737406225356574 Y[1,9] + 0.7719163558447772 Y[2,9] + 0.37738325943275364 Y[3,9] + 0.8755549995596186 Y[1,10] + 0.26456814036193066 Y[2,10] + 0.9018773245304149 Y[3,10]

In [42]:
optimize!(model)
status = JuMP.termination_status(model)
# solution_summary(model)
if status == MOI.OPTIMAL
    obj_val = objective_value(model)
    X_val = value.(X)
else
    obj_val = NaN
    X_val = ones(N,N) .* NaN
end

Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 654             
  Affine conic cons.     : 6 (18 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 204             
  Matrix variables       : 0               
  Integer variables      : 30              

Optimizer started.
Mixed integer optimizer started.
Threads used: 8
Presolve started.
Presolve terminated. Time = 0.00, probing time =  0.00
Presolved problem: 198 variables, 468 constraints, 1238 non-zeros
Presolved problem: 0 general integer, 30 binary, 168 continuous
Presolved problem: 6 cones
Clique table size: 3
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   2.5760574846e+00     NA          0.0   
0        1        1      

3×10 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0  -0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   0.0  -0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -0.0   0.0

In [43]:
price = sum(X_val .* p_dag,dims=2)[:,1]
utilities = ones(N+1)
utilities[1:N] = exp.(vec(A * u + B * price))

prob = zeros(N)
rev = zeros(N)
for i in 1:N
    prob[i] = utilities[i] / sum(utilities)
    rev[i] = prob[i] * price[i]
end
println("rev =",sum(rev))
println("price =", price)

rev =0.6883497210244508
price =[0.884629777785838, 0.9111498653643074, 0.9158663552785268]


# The second reformulation

In [ ]:
model = Model(Mosek.Optimizer)
# set_attribute(model, "QUIET", true)
# model = Model(COPT.ConeOptimizer)
# 定义变量
@variable(model, rho_0 >= 0)                       # ρ₀ ≥ 0
@variable(model, rho[1:N] >= 0)                    # ρ_n ≥ 0
@variable(model, alpha[1:N,1:N_u])                   # ς = ρ₀ * u
@variable(model, beta[1:N,1:N])                       # φ_n
@variable(model, Y[1:N, 1:K])                      # y_{nk}
@variable(model, Z[1:N, 1:N, 1:K])                      # z_{nk}
@variable(model, X[1:N, 1:K], Bin)                 # x_{nk} ∈ {0,1}

In [ ]:
@constraint(model, rho_0 + sum(rho) == 1)

In [ ]:
for n in 1:N
    @constraint(model, [rho[n], rho_0, A[n,:]' * u + sum(Z[n,:,:] .* p_dag)] in MOI.ExponentialCone())
end

In [ ]:
@constraint(model, alpha .- rho_0 .* A .== 0)

In [ ]:
@constraint(model, beta .- rho_0 .* B .== 0)

In [ ]:
for n in 1:N
    for k in 1:K
        # y_{nk} bounds
        @constraint(model, 0 <= Y[n, k])
        @constraint(model, Y[n, k] <= X[n, k])
        @constraint(model, Y[n, k] >= rho[n] - (1 - X[n, k]))
        @constraint(model, Y[n, k] <= rho[n])
    end
end

In [ ]:
beta_lb = -100
beta_ub = 100
for n in 1:N
    for j in 1:N
        for k in 1:K
            # y_{nk} bounds
            @constraint(model, beta_lb * X[j,k] <= Z[n,j,k])
            @constraint(model, Z[n, j, k] <= beta_ub * X[j, k])
            @constraint(model, Z[n, j, k] >= beta[n,j] - beta_ub * (1 - X[j, k]))
            @constraint(model, Z[n, j, k] <= beta[n,j] - beta_lb * (1 - X[j, k]))
        end
    end
end

In [ ]:
@constraint(model, sum(X,dims=2) .== 1)

In [ ]:
@objective(model, Min,sum(Y .* p_dag) - tau * sum(rho))

In [ ]:
optimize!(model)
status = JuMP.termination_status(model)
# solution_summary(model)
if status == MOI.OPTIMAL
    obj_val = objective_value(model)
    X_val = value.(X)
else
    obj_val = NaN
    X_val = ones(N,N) .* NaN
end